In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import OneClassSVM
from utils import AudioSVMClassifier
import os

In [2]:
base_directory = "/data/Deep_Fake_Data/"
feat_directory = "Features_no_padding"
data_names = 'ASV19'
data_types = "train"
features_ls  = ['ssast_patch_base', 'wavlm_large', 'data2vec_large_ll60k', 'wavlablm_mk_40k', 'unispeech_sat_large', 'hubert_large_ll60k', 'xlsr_53', 'passt_base', 'espnet_hubert_base_iter0', 'mae_ast_patch', 'ast', 'byol_a_2048', 'modified_cpc', 
                 'byol_s_cvt', 'wav2vec2_base_s2st_es_voxpopuli']
speakers_ls = ['p229']
deepfake_types  = ['A01']
output_dir = 'one_class_svm'

In [3]:
nu=0.05
gamma='auto'

In [4]:
for speaker in speakers_ls:
    for deepfake in deepfake_types:
        for feature in features_ls:
            path_to_original_data = os.path.join(base_directory, feat_directory, data_names, speaker, data_types, 'Original', feature)
            classifier = AudioSVMClassifier(speaker, base_directory, feature, output_dir)
            classifier.train(path_to_original_data, nu, gamma)
            path_to_deepfake_data = os.path.join(base_directory, feat_directory, data_names, speaker, data_types, deepfake, feature)
            y_orig, y_pred = classifier.test(path_to_deepfake_data, )
            results = classifier.evaluate_model(compute_accuracy=True, compute_auc=True, compute_conf_matrix=False)


one_class_svm/p229
one_class_svm/p229


/home/selloh/UMAP/venv2/lib64/python3.9/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/selloh/UMAP/venv2/lib64/python3.9/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/selloh/UMAP/venv2/lib64/python3.9/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/selloh/UMAP/venv2/lib64/python3.9/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/selloh/UMAP/venv2/lib64/python3.9/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warni

one_class_svm/p229
one_class_svm/p229
one_class_svm/p229
one_class_svm/p229
one_class_svm/p229


/home/selloh/UMAP/venv2/lib64/python3.9/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/selloh/UMAP/venv2/lib64/python3.9/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/selloh/UMAP/venv2/lib64/python3.9/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/selloh/UMAP/venv2/lib64/python3.9/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(


one_class_svm/p229
one_class_svm/p229
one_class_svm/p229


/home/selloh/UMAP/venv2/lib64/python3.9/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/selloh/UMAP/venv2/lib64/python3.9/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/selloh/UMAP/venv2/lib64/python3.9/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/selloh/UMAP/venv2/lib64/python3.9/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(


one_class_svm/p229
one_class_svm/p229
one_class_svm/p229
one_class_svm/p229
one_class_svm/p229


/home/selloh/UMAP/venv2/lib64/python3.9/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/selloh/UMAP/venv2/lib64/python3.9/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
